<a href="https://colab.research.google.com/github/EssenceBL/MakingFriends/blob/main/Study_Notes_A_Extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Foreword**

This series of Colab notebooks contains study notes related to Descartes' Circle Theorem. The motivation for this work originated from attending Professor Peter Sarnak's Shaw Prize Lecture in Mathematical Sciences in 2024. Not only does his lecture begin with Descartes' Circle Theorem, but he also provides significant insights into the importance of establishing a "base" for one’s long-term endeavors. Reflecting on his remarks, I believe that exploring mathematical concepts through Python code could serve as my "base," drawing from prior experiences; thus, I produced this series of Colab notebooks. Some of the code comes from iterative collaboration with DeepSeek, representing an experiment that integrates traditional mathematical mindsets with contemporary computational methodologies.

YK with DeepSeek, 2025

Like other Colab notebooks, please run the code cells one by one. In particular, the following code cell concerns the dependency packages and needs to be run first. It might take a while, so please be patient.

**Extension: An Interactive Game**

We review from the relation $(b_1+b_2+b_3)^2=2(b_1^2+b_2^2+b_3^2)$ that describes the curvature of three mutually tangent circles with a common tangent line. Here We may observe more interesting phenomenon in the number theoretical aspect.

We may regard the relation as a quadratic equation in $b_3$ and solve for it from $b_1$ and $b_2$:

\begin{eqnarray}
(b_1+b_2+b_3)^2&=&2(b_1^2+b_2^2+b_3^2)\\
b_3^2-2(b_1+b_2)b_3+(b_1-b_2)^2&=&0\\
b_3&=&b_1+b_2\pm 2\sqrt{b_1b_2}\\
b_3&=&(\sqrt{b_1}\pm\sqrt{b_2})^2
\end{eqnarray}

There are two solutions: one corresponds to inscribing the third circle between the previous two circles, whereas the other corresponds to excribing the third circle outside of the previous circles.

From the expression $b_3=(\sqrt{b_1}\pm\sqrt{b_2})^2$, it is easy to observe that, if both $b_1$ and $b_2$ are perfect square, so are the two solutions of $b_3$. This generalizes the previous calculation in the special case with $b_1$ and $b_2$ are consecutive integers.

This allows us to do some iterations related to something called a Stern–Brocot tree with some scaling under a more careful look. Instead of going into further details, we will just enjoy the following interactive python game in the school workshop. The codes are again a result of our pair-programming experiment with DeepSeek.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

def parse_input_stern_brocot(input_str):
    current_frac = (1, 2)
    current_k = 4
    lp_frac, l_k = (0, 1), 1
    rp_frac, r_k = (1, 1), 1

    for c in input_str:
        if c == 'r':
            new_frac = (current_frac[0] + rp_frac[0], current_frac[1] + rp_frac[1])
            new_lp_frac, new_l_k = current_frac, current_k
            new_rp_frac, new_r_k = rp_frac, r_k
        elif c == 'l':
            new_frac = (lp_frac[0] + current_frac[0], lp_frac[1] + current_frac[1])
            new_lp_frac, new_l_k = lp_frac, l_k
            new_rp_frac, new_r_k = current_frac, current_k
        else:
            raise ValueError("Invalid character. Use '+' or '-'.")

        new_k = new_l_k + new_r_k + 2 * np.sqrt(new_l_k * new_r_k)
        lp_frac, l_k = new_lp_frac, new_l_k
        rp_frac, r_k = new_rp_frac, new_r_k
        current_frac, current_k = new_frac, new_k

    return {
        'frac': current_frac,
        'lp_frac': lp_frac, 'l_k': l_k,
        'rp_frac': rp_frac, 'r_k': r_k,
        'k': current_k
    }

def generate_iterations(target_node, iterations):
    all_nodes = [target_node]
    current_gen = [target_node]

    for _ in range(iterations):
        next_gen = []
        for node in current_gen:
            # Left child
            l_frac = (node['lp_frac'][0] + node['frac'][0], node['lp_frac'][1] + node['frac'][1])
            l_k = node['l_k'] + node['k'] + 2 * np.sqrt(node['l_k'] * node['k'])
            next_gen.append({
                'frac': l_frac,
                'lp_frac': node['lp_frac'], 'l_k': node['l_k'],
                'rp_frac': node['frac'], 'r_k': node['k'],
                'k': l_k
            })
            # Right child
            r_frac = (node['frac'][0] + node['rp_frac'][0], node['frac'][1] + node['rp_frac'][1])
            r_k = node['k'] + node['r_k'] + 2 * np.sqrt(node['k'] * node['r_k'])
            next_gen.append({
                'frac': r_frac,
                'lp_frac': node['frac'], 'l_k': node['k'],
                'rp_frac': node['rp_frac'], 'r_k': node['r_k'],
                'k': r_k
            })
        all_nodes.extend(next_gen)
        current_gen = next_gen
    return all_nodes

def plot_generated_circles(nodes, target_node):
    plt.figure(figsize=(12, 8))
    ax = plt.gca()
    x_coords = []
    radii = []
    for node in nodes:
        x = 2*node['frac'][0] / node['frac'][1]
        radius = 1 / node['k']
        x_coords.append(x)
        radii.append(radius)
        circle = Circle((x, radius), radius, fill=False, edgecolor='blue', linewidth=0.5)
        ax.add_patch(circle)
        plt.text(x, radius, f"{node['k']}", ha='center', va='center', color='blue', fontsize=200*radius*0.25/max(radii))

    x_min = min(x_coords)
    x_max = max(x_coords)
    y_max = max(radii) * 3

    print("5 iterations from a circle with curvature ", target_node['k'])

    lParentx = 2*target_node['lp_frac'][0] / target_node['lp_frac'][1]
    lParentRadius = 1 / target_node['l_k']
    x_coords.append(lParentx)
    radii.append(lParentRadius)
    circle = Circle((lParentx, lParentRadius), lParentRadius, fill=False, edgecolor='blue', linewidth=0.5)
    ax.add_patch(circle)
    #plt.text(lParentx, lParentRadius, f"{target_node['l_k']}", ha='center', va='center', color='red', fontsize=12)

    rParentx = 2*target_node['rp_frac'][0] / target_node['rp_frac'][1]
    rParentRadius = 1 / target_node['r_k']
    x_coords.append(rParentx)
    radii.append(rParentRadius)
    circle = Circle((rParentx, rParentRadius), rParentRadius, fill=False, edgecolor='blue', linewidth=0.5)
    ax.add_patch(circle)
    #plt.text(rParentx, rParentRadius, f"{target_node['r_k']}", ha='center', va='center', color='red', fontsize=12)


    #x_min = lParentx-lParentRadius
    #x_max = rParentx+rParentRadius

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0, y_max)
    ax.set_aspect('equal')
    ax.set_title("5 iterations from the assoicated circle")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(False)
    plt.show()

def stern_brocot_plot(input_str):
    try:
        target_node = parse_input_stern_brocot(input_str)
        all_nodes = generate_iterations(target_node, iterations=5)
        plot_generated_circles(all_nodes, target_node)
    except Exception as e:
        print(f"Error: {str(e)}")

input_widget = widgets.Text(
    value='',
    #value='rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr',
    #value='rrrrrrrrrrlll',
    #value='rrrrlrlr',
    placeholder='consists of r and l (e.g. rrrrlrlr)',
    description='Associated circle:',
    style={'description_width': 'initial'}
)

widgets.interactive(stern_brocot_plot, input_str=input_widget)
